In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./input/train_identity.csv
./input/sample_submission.csv
./input/test_transaction.csv
./input/train_transaction.csv
./input/test_identity.csv


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./output'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./output/test_transaction_update_1.pkl
./output/test_transaction_update.pkl
./output/train_transaction_update_1.pkl
./output/train_transaction_update.pkl
./output/last_transaction.pkl
./output/transaction_identity.pkl
./output/Groups.pkl


In [3]:
import matplotlib.pyplot as plt
import datetime
import decimal
import seaborn as sns
import warnings 
import gc,os,random
from decimal import *
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
warnings.filterwarnings(action='ignore')
%matplotlib inline
pd.options.display.max_rows=500
pd.options.display.max_columns=100
pd.options.display.max_colwidth=100
gc.enable()
dates_range = pd.date_range(start='2017-10-01', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,GroupKFold,StratifiedKFold
import lightgbm as lgb
from tqdm import tqdm_notebook as tqdm
from sklearn import metrics

In [4]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
SEED = 42
seed_everything(SEED)
LOCAL_TEST = False
TARGET = 'isFraud'

In [5]:
train = pd.read_csv('./input/train_transaction.csv')
test = pd.read_csv('./input/test_transaction.csv')
test['isFraud']=0
nrows = train.shape[0]

In [6]:
def values_normalization(dt_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            dt_df[col] = dt_df[col].astype(float)  

            temp_min = dt_df.groupby([period])[col].agg(['min']).reset_index()
            temp_min.index = temp_min[period].values
            temp_min = temp_min['min'].to_dict()

            temp_max = dt_df.groupby([period])[col].agg(['max']).reset_index()
            temp_max.index = temp_max[period].values
            temp_max = temp_max['max'].to_dict()

            temp_mean = dt_df.groupby([period])[col].agg(['mean']).reset_index()
            temp_mean.index = temp_mean[period].values
            temp_mean = temp_mean['mean'].to_dict()

            temp_std = dt_df.groupby([period])[col].agg(['std']).reset_index()
            temp_std.index = temp_std[period].values
            temp_std = temp_std['std'].to_dict()

            dt_df['temp_min'] = dt_df[period].map(temp_min)
            dt_df['temp_max'] = dt_df[period].map(temp_max)
            dt_df['temp_mean'] = dt_df[period].map(temp_mean)
            dt_df['temp_std'] = dt_df[period].map(temp_std)

            dt_df[new_col+'_min_max'] = (dt_df[col]-dt_df['temp_min'])/(dt_df['temp_max']-dt_df['temp_min'])
            dt_df[new_col+'_std_score'] = (dt_df[col]-dt_df['temp_mean'])/(dt_df['temp_std'])
            del dt_df['temp_min'],dt_df['temp_max'],dt_df['temp_mean'],dt_df['temp_std']
    return dt_df

def frequency_encoding(train_df, test_df, columns, self_encoding=False):
    for col in columns:
        temp_df = pd.concat([train_df[[col]], test_df[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()
        if self_encoding:
            train_df[col] = train_df[col].map(fq_encode)
            test_df[col]  = test_df[col].map(fq_encode)            
        else:
            train_df[col+'_fq_enc'] = train_df[col].map(fq_encode)
            test_df[col+'_fq_enc']  = test_df[col].map(fq_encode)
    return train_df, test_df


def timeblock_frequency_encoding(train_df, test_df, periods, columns, 
                                 with_proportions=True, only_proportions=False):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            train_df[new_col] = train_df[col].astype(str)+'_'+train_df[period].astype(str)
            test_df[new_col]  = test_df[col].astype(str)+'_'+test_df[period].astype(str)

            temp_df = pd.concat([train_df[[new_col]], test_df[[new_col]]])
            fq_encode = temp_df[new_col].value_counts().to_dict()

            train_df[new_col] = train_df[new_col].map(fq_encode)
            test_df[new_col]  = test_df[new_col].map(fq_encode)
            
            if only_proportions:
                train_df[new_col] = train_df[new_col]/train_df[period+'_total']
                test_df[new_col]  = test_df[new_col]/test_df[period+'_total']

            if with_proportions:
                train_df[new_col+'_proportions'] = train_df[new_col]/train_df[period+'_total']
                test_df[new_col+'_proportions']  = test_df[new_col]/test_df[period+'_total']

    return train_df, test_df


def uid_aggregation(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name]  = test_df[col].map(temp_df)
    return train_df, test_df


def uid_aggregation_and_normalization(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:  
        for col in uids:
            
            new_norm_col_name = col+'_'+main_column+'_std_norm'
            norm_cols = []
            
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name]  = test_df[col].map(temp_df)
                norm_cols.append(new_col_name)
            
            train_df[new_norm_col_name] = (train_df[main_column]-train_df[norm_cols[0]])/train_df[norm_cols[1]]
            test_df[new_norm_col_name]  = (test_df[main_column]-test_df[norm_cols[0]])/test_df[norm_cols[1]]          
            
            del train_df[norm_cols[0]], train_df[norm_cols[1]]
            del test_df[norm_cols[0]], test_df[norm_cols[1]]
                                              
    return train_df, test_df

## DateTime Feature

In [9]:
# TransactionDateTime
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
train['TransactionDateTime'] = train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
test['TransactionDateTime'] = test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
date_cols = ['TransactionDateTime']
for date_col in date_cols:
    train[date_col + "_month"] = train[date_col].dt.month
    test[date_col + "_month"] = test[date_col].dt.month
    
    train[date_col + "_hour"] = train[date_col].dt.hour
    test[date_col + "_hour"] = test[date_col].dt.hour
    
    train[date_col + "_year"] = train[date_col].dt.year
    test[date_col + "_year"] = test[date_col].dt.year
    
    train[date_col + "_week"] = train[date_col].dt.week
    test[date_col + "_week"] = test[date_col].dt.week
    
    train[date_col + "_weekDay"] = train[date_col].dt.weekday
    test[date_col + "_weekDay"] = test[date_col].dt.weekday
    
    train[date_col + "_day"] = train[date_col].dt.day
    test[date_col + "_day"] = test[date_col].dt.day
train['is_december'] = train['TransactionDateTime'].dt.month
train['is_december'] = (train['is_december']==12).astype(np.int8)
test['is_december'] = test['TransactionDateTime'].dt.month
test['is_december'] = (test['is_december']==12).astype(np.int8)

# Holidays
train['is_holiday'] = (train['TransactionDateTime'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
test['is_holiday'] = (test['TransactionDateTime'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)


for col in ['TransactionDateTime_month','TransactionDateTime_week','TransactionDateTime_day','is_holiday']:
    temp = pd.concat([train[[col]], test[[col]]])
    fq_encode = temp[col].value_counts().to_dict()
            
    train[col + '_total'] = train[col].map(fq_encode)
    test[col + '_total']  = test[col].map(fq_encode)

In [10]:
del temp
gc.collect()

31

### uid merge

In [11]:
train['uid'] = train['card1'].astype(str)+'_'+train['card2'].astype(str)
test['uid'] = test['card1'].astype(str)+'_'+test['card2'].astype(str)

train['uid2'] = train['uid'].astype(str)+'_'+train['card3'].astype(str)+'_'+train['card5'].astype(str)
test['uid2'] = test['uid'].astype(str)+'_'+test['card3'].astype(str)+'_'+test['card5'].astype(str)

train['uid3'] = train['uid2'].astype(str)+'_'+train['card4'].astype(str)
test['uid3'] = test['uid2'].astype(str)+'_'+test['card4'].astype(str)

train['uid4'] = train['uid3'].astype(str)+'_'+train['card6'].astype(str)
test['uid4'] = test['uid3'].astype(str)+'_'+test['card6'].astype(str)

train['uid5'] = train['uid4'].astype(str)+'_'+train['addr1'].astype(str)+'_'+train['addr2'].astype(str)
test['uid5'] = test['uid4'].astype(str)+'_'+test['addr1'].astype(str)+'_'+test['addr2'].astype(str)

train['uid6'] = train['uid5'].astype(str)+'_'+train['P_emaildomain'].astype(str)
test['uid6'] = test['uid5'].astype(str)+'_'+test['P_emaildomain'].astype(str)

train['uid7'] = train['uid6'].astype(str)+'_'+train['R_emaildomain'].astype(str)
test['uid7'] = test['uid6'].astype(str)+'_'+test['R_emaildomain'].astype(str)

train['bank_type'] = train['card3'].astype(str)+'_'+train['card5'].astype(str)
test['bank_type'] = test['card3'].astype(str)+'_'+test['card5'].astype(str)




new_columns = ['uid','uid2','uid3','uid4','uid5','uid6','uid7','bank_type']



i_cols = ['card1','card2','card3','card4','card5','card6','addr1','addr2','P_emaildomain','R_emaildomain'] + new_columns
train , test = frequency_encoding(train, test, i_cols, self_encoding=False)

## Decimal part split

In [12]:
## TransactionAmt
def Decimal_split(x):
    number = round(Decimal(x)-int(x),3)*1000
    return int(number)

def Decimal_Boolean(x):
    if x==0:
        return 0
    else:
        return 1

def Decimal_len(x):
    number = round(Decimal(x)-int(x),3)*1000
    return len(str(int(number)))
    

    
train['TransactionAmt_decimal'] = np.round(train['TransactionAmt'] - np.floor(train['TransactionAmt']),2)
test['TransactionAmt_decimal'] = np.round(test['TransactionAmt'] - np.floor(test['TransactionAmt']),2 )


train['TransactionAmt_decimal_bool'] = train['TransactionAmt_decimal'].apply(Decimal_Boolean)
test['TransactionAmt_decimal_bool'] = test['TransactionAmt_decimal'].apply(Decimal_Boolean)

train['TransactionAmt_decimal_length'] = train['TransactionAmt'].apply(Decimal_len)
test['TransactionAmt_decimal_length'] = test['TransactionAmt'].apply(Decimal_len)

train['TransactionAmt_Integer'] = train['TransactionAmt'].astype(np.int32)
test['TransactionAmt_Integer'] = test['TransactionAmt'].astype(np.int32)


mask = train['TransactionAmt_decimal']==0
train.loc[mask,'TransactionAmt_decimal_length']=0

mask = test['TransactionAmt_decimal']==0
test.loc[mask,'TransactionAmt_decimal_length']=0

col = "TransactionAmt_decimal"

temp_df = pd.concat([train[[col]], test[[col]]])
valid_card = temp_df[col].value_counts().to_dict()   
train[col+'_fq_enc'] = train[col].map(valid_card)
test[col+'_fq_enc']  = test[col].map(valid_card)



col=['ProductCD','TransactionAmt_decimal']
temp_df = pd.concat([train[col], test[col]])
_ = temp_df.groupby(['ProductCD','TransactionAmt_decimal']).size().reset_index()
_.columns=['ProductCD','TransactionAmt_decimal','ProductCD_TransactionAmt_decimal_Size']
train = pd.merge(train,_,on=['ProductCD','TransactionAmt_decimal'],how='left')
test = pd.merge(test,_,on=['ProductCD','TransactionAmt_decimal'],how='left')

_ = temp_df.groupby(['ProductCD'])['TransactionAmt_decimal'].nunique().reset_index()
_.columns=['ProductCD','ProductCD_TransactionAmt_decimal_nunique']
train = pd.merge(train,_,on=['ProductCD'],how='left')
test = pd.merge(test,_,on=['ProductCD'],how='left')



del _,temp_df
gc.collect()

0

### productCD and decimal integer part addition

In [13]:
train['product_type'] = train['ProductCD'].astype(str)+'_'+train['TransactionAmt'].astype(str)
test['product_type'] = test['ProductCD'].astype(str)+'_'+test['TransactionAmt'].astype(str)
train['product_type2'] = train['ProductCD'].astype(str)+'_'+train['TransactionAmt_decimal'].astype(str)
test['product_type2'] = test['ProductCD'].astype(str)+'_'+test['TransactionAmt_decimal'].astype(str)

periods = ['TransactionDateTime_month','TransactionDateTime_week','TransactionDateTime_day']
i_cols = ['product_type', 'bank_type', "product_type2"]


In [16]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,...,is_december,is_holiday,TransactionDateTime_month_total,TransactionDateTime_week_total,TransactionDateTime_day_total,is_holiday_total,uid,uid2,uid3,uid4,uid5,uid6,uid7,bank_type,card1_fq_enc,card2_fq_enc,card3_fq_enc,card4_fq_enc,card5_fq_enc,card6_fq_enc,addr1_fq_enc,addr2_fq_enc,P_emaildomain_fq_enc,R_emaildomain_fq_enc,uid_fq_enc,uid2_fq_enc,uid3_fq_enc,uid4_fq_enc,uid5_fq_enc,uid6_fq_enc,uid7_fq_enc,bank_type_fq_enc,TransactionAmt_decimal,TransactionAmt_decimal_bool,TransactionAmt_decimal_length,TransactionAmt_Integer,TransactionAmt_decimal_fq_enc,ProductCD_TransactionAmt_decimal_Size,ProductCD_TransactionAmt_decimal_nunique,product_type,product_type2,product_type_TransactionDateTime_month,bank_type_TransactionDateTime_month,product_type2_TransactionDateTime_month,product_type_TransactionDateTime_week,bank_type_TransactionDateTime_week,product_type2_TransactionDateTime_week,product_type_TransactionDateTime_day,bank_type_TransactionDateTime_day,product_type2_TransactionDateTime_day
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,...,1,0,254521,30284,40309,1061762,13926_nan,13926_nan_150.0_142.0,13926_nan_150.0_142.0_discover,13926_nan_150.0_142.0_discover_credit,13926_nan_150.0_142.0_discover_credit_315.0_87.0,13926_nan_150.0_142.0_discover_credit_315.0_87.0_nan,13926_nan_150.0_142.0_discover_credit_315.0_87.0_nan_nan,150.0_142.0,56,17587,956845,9524,309,267648,43035,956415,163648,824070,6,6,6,6,2,1,1,309,0.5,1,3,68,33877,32687,100,W_68.5,W_0.5,0.000841,0.000130,0.021943,0.001222,0.000066,0.023115,0.001290,0.000248,0.030812
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,...,1,0,254521,30284,40309,1061762,2755_404.0,2755_404.0_150.0_102.0,2755_404.0_150.0_102.0_mastercard,2755_404.0_150.0_102.0_mastercard_credit,2755_404.0_150.0_102.0_mastercard_credit_325.0_87.0,2755_404.0_150.0_102.0_mastercard_credit_325.0_87.0_gmail.com,2755_404.0_150.0_102.0_mastercard_credit_325.0_87.0_gmail.com_nan,150.0_102.0,1338,5593,956845,347386,49491,267648,76902,956415,435803,824070,1328,1328,1328,1328,105,21,19,44082,0.0,0,0,29,573420,413744,100,W_29.0,W_0.0,0.011928,0.043937,0.281843,0.015586,0.036488,0.390966,0.017614,0.029249,0.409239
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,T,T,T,M0,...,1,0,254521,30284,40309,1061762,4663_490.0,4663_490.0_150.0_166.0,4663_490.0_150.0_166.0_visa,4663_490.0_150.0_166.0_visa_debit,4663_490.0_150.0_166.0_visa_debit_330.0_87.0,4663_490.0_150.0_166.0_visa_debit_330.0_87.0_outlook.com,4663_490.0_150.0_166.0_visa_debit_330.0_87.0_outlook.com_nan,150.0_166.0,1794,70496,956845,719649,102930,824959,48387,956415,9934,824070,1787,1787,1787,1787,46,4,4,101292,0.0,0,0,59,573420,413744,100,W_59.0,W_0.0,0.035247,0.071527,0.281843,0.044809,0.103916,0.390966,0.053090,0.141234,0.409239
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN,M0,...,1,0,254521,30284,40309,1061762,18132_567.0,18132_567.0_150.0_117.0,18132_567.0_150.0_117.0_mastercard,18132_567.0_150.0_117.0_mastercard_debit,18132_567.0_150.0_117.0_mastercard_debit_476.0_87.0,18132_567.0_150.0_117.0_mastercar

In [15]:
train, test = timeblock_frequency_encoding(train, test, periods, i_cols, with_proportions=False, only_proportions=True)

### C columns

In [18]:
c_cols = ['C'+str(i) for i in range(1,15)]

## Sum of Null values in C columns
train['sum_of_C_columns'] = train[c_cols].sum(axis=1)
test['sum_of_C_columns'] = test[c_cols].sum(axis=1)

## Sum of 0 values in C columns
mask = train[c_cols]==0
mask1 = test[c_cols]==0
train['no_of_zeros_in_C_Columns'] = mask.sum(axis=1)
test['no_of_zeros_in_C_Columns'] = mask1.sum(axis=1)

## Sum of >0 values in C columns
train['greater_than_zeros_No_of_C_Columns'] = 14 - train['no_of_zeros_in_C_Columns']
test['greater_than_zeros_No_of_C_Columns'] = 14 - test['no_of_zeros_in_C_Columns']
gc.collect()


## max values in C columns

train['maximum_values_in_C_Columns'] = train[c_cols].max(axis=1)
test['maximum_values_in_C_Columns'] = test[c_cols].max(axis=1)

del mask,mask1
gc.collect()


28


### M Columns

In [19]:
m_cols = ['M'+str(i) for i in range(1,10)]
## sum of null values from M1-M9
train['Null_Values_M_Columns'] = train[m_cols].isnull().sum(axis=1)
test['Null_Values_M_Columns'] = test[m_cols].isnull().sum(axis=1)

## sum of T values from M1-M9
mask = train[m_cols]=='T'
mask1 = test[m_cols]=='T'
train['T_values_M_Columns'] = mask.sum(axis=1)
test['T_values_M_Columns'] = mask1.sum(axis=1)

## sum of F values from M1-M9
mask = train[m_cols]=='F'
mask1 = test[m_cols]=='F'
train['F_values_M_Columns'] = mask.sum(axis=1)
test['F_values_M_Columns'] = mask1.sum(axis=1)
gc.collect()



m_cols = ['M1','M2','M3']
## sum of null values from M1-M3
train['Null_Values_M1_to_M3_Columns'] = train[m_cols].isnull().sum(axis=1)
test['Null_Values_M1_to_M3_Columns'] = test[m_cols].isnull().sum(axis=1)

## sum of T values from M1-M3
mask = train[m_cols]=='T'
mask1 = test[m_cols]=='T'
train['T_values_M1_to_M3_Columns'] = mask.sum(axis=1)
test['T_values_M1_to_M3_Columns'] = mask1.sum(axis=1)

## sum of F values from M1-M3
mask = train[m_cols]=='F'
mask1 = test[m_cols]=='F'
train['F_values_M1_to_M3_Columns'] = mask.sum(axis=1)
test['F_values_M1_to_M3_Columns'] = mask1.sum(axis=1)
gc.collect()
         
    

m_cols = ['M7','M8','M9']
## sum of null values from M7-M9
train['Null_Values_M7_to_M9_Columns'] = train[m_cols].isnull().sum(axis=1)
test['Null_Values_M7_to_M9_Columns'] = test[m_cols].isnull().sum(axis=1)

## sum of T values from M7-M7
mask = train[m_cols]=='T'
mask1 = test[m_cols]=='T'
train['T_values_M7_to_M9_Columns'] = mask.sum(axis=1)
test['T_values_M7_to_M9_Columns'] = mask1.sum(axis=1)

## sum of F values from M7-M9
mask = train[m_cols]=='F'
mask1 = test[m_cols]=='F'
train['F_values_M7_to_M9_Columns'] = mask.sum(axis=1)
test['F_values_M7_to_M9_Columns'] = mask1.sum(axis=1)
gc.collect()



train['M4_isNull']=0
test['M4_isNull']=0
mask = train['M4'].isnull()
train.ix[mask,'M4_isNull']=1

mask = test['M4'].isnull()
test.ix[mask,'M4_isNull']=1


del mask,mask1
gc.collect()

28

### D Columns

In [20]:
d_cols = ['D'+str(i) for i in range(1,16)]

train.ix[train['D15']<0,'D15'] = abs(train[train['D15']<0]['D15'].values)
train.ix[train['D14']<0,'D14'] = abs(train[train['D14']<0]['D14'].values)
train.ix[train['D12']<0,'D12'] = abs(train[train['D12']<0]['D12'].values)
train.ix[train['D11']<0,'D11'] = abs(train[train['D11']<0]['D11'].values)
train.ix[train['D6']<0,'D6'] = abs(train[train['D6']<0]['D6'].values)
train.ix[train['D4']<0,'D4'] = abs(train[train['D4']<0]['D4'].values)


test.ix[test['D15']<0,'D15'] = abs(test[test['D15']<0]['D15'].values)
test.ix[test['D14']<0,'D14'] = abs(test[test['D14']<0]['D14'].values)
test.ix[test['D12']<0,'D12'] = abs(test[test['D12']<0]['D12'].values)
test.ix[test['D11']<0,'D11'] = abs(test[test['D11']<0]['D11'].values)
test.ix[test['D6']<0,'D6'] = abs(test[test['D6']<0]['D6'].values)
test.ix[test['D4']<0,'D4'] = abs(test[test['D4']<0]['D4'].values)

In [21]:
null = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14','D15']
for col in null:
    print(f'{col} : {train[train[col]<0][col].shape}')

D1 : (0,)
D2 : (0,)
D3 : (0,)
D4 : (0,)
D5 : (0,)
D6 : (0,)
D7 : (0,)
D8 : (0,)
D9 : (0,)
D10 : (0,)
D11 : (0,)
D12 : (0,)
D13 : (0,)
D14 : (0,)
D15 : (0,)


In [22]:
null = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14','D15']
for col in null:
    print(f'{col} : {test[test[col]<0][col].shape}')

D1 : (0,)
D2 : (0,)
D3 : (0,)
D4 : (0,)
D5 : (0,)
D6 : (0,)
D7 : (0,)
D8 : (0,)
D9 : (0,)
D10 : (0,)
D11 : (0,)
D12 : (0,)
D13 : (0,)
D14 : (0,)
D15 : (0,)


In [23]:
for df in [train, test]:
    # Lets transform D8 and D9 column
    # As we almost sure it has connection with hours
    df['D9_not_na'] = np.where(df['D9'].isna(),0,1)
    df['D8_not_same_day'] = np.where(df['D8']>=1,1,0)
    df['D8_D9_decimal_dist'] = df['D8'].fillna(0)-df['D8'].fillna(0).astype(int)
    df['D8_D9_decimal_dist'] = ((df['D8_D9_decimal_dist']-df['D9'])**2)**0.5

In [24]:
gc.collect()

61

### Email_Domain

In [25]:
## same email_domain present in both P,Remaildomain
train['Email_Present_in_PRDomain']=0
mask = train['P_emaildomain'].isna()
mask1 = train['R_emaildomain'].isna()
train.ix[train['P_emaildomain']==train['R_emaildomain'],'Email_Present_in_PRDomain']=1
train.ix[(train['P_emaildomain']!=train['R_emaildomain'])&(~mask)&(~mask1),'Email_Present_in_PRDomain']=2
train.ix[(~mask)&(mask1),'Email_Present_in_PRDomain']=3
train.ix[(~mask1)&(mask),'Email_Present_in_PRDomain']=4

train['Both_Email_isNull']=0
train.ix[(mask)&(mask1),'Both_Email_isNull']=1


## same email_domain present in both P,Remaildomain
test['Email_Present_in_PRDomain']=0
mask = test['P_emaildomain'].isna()
mask1 = test['R_emaildomain'].isna()
test.ix[test['P_emaildomain']==test['R_emaildomain'],'Email_Present_in_PRDomain']=1
test.ix[(test['P_emaildomain']!=test['R_emaildomain'])&(~mask)&(~mask1),'Email_Present_in_PRDomain']=2
test.ix[(~mask)&(mask1),'Email_Present_in_PRDomain']=3
test.ix[(~mask1)&(mask),'Email_Present_in_PRDomain']=4


test['Both_Email_isNull']=0
test.ix[(mask)&(mask1),'Both_Email_isNull']=1


train['Email_Present_in_PRDomain_Full_Count'] = train['Email_Present_in_PRDomain'].map(pd.concat([train['Email_Present_in_PRDomain'],test['Email_Present_in_PRDomain']],sort=False,ignore_index=True).value_counts(dropna=False))
test['Email_Present_in_PRDomain_Full_Count'] = test['Email_Present_in_PRDomain'].map(pd.concat([train['Email_Present_in_PRDomain'],test['Email_Present_in_PRDomain']],sort=False,ignore_index=True).value_counts(dropna=False))

train['Both_Email_isNull_Full_Count'] = train['Both_Email_isNull'].map(pd.concat([train['Both_Email_isNull'],test['Both_Email_isNull']],sort=False,ignore_index=True).value_counts(dropna=False))
test['Both_Email_isNull_Full_Count'] = test['Both_Email_isNull'].map(pd.concat([train['Both_Email_isNull'],test['Both_Email_isNull']],sort=False,ignore_index=True).value_counts(dropna=False))

In [26]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

In [27]:
for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c].map(emails)
    test[c + '_bin'] = test[c].map(emails)
    
    train[c + '_suffix'] = train[c].map(lambda x: str(x).split('.')[-1])
    test[c + '_suffix'] = test[c].map(lambda x: str(x).split('.')[-1])
    
    train[c + '_suffix'] = train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test[c + '_suffix'] = test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

## Freq Encoding

In [28]:
    cols = ['dist1','dist2','ProductCD',
           'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14',
           'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14','D15']
    for col in cols:
        temp_df = pd.concat([train[[col]], test[[col]]])
        valid_card = temp_df[col].value_counts().to_dict()   
        train[col+'_fq_enc'] = train[col].map(valid_card)
        test[col+'_fq_enc']  = test[col].map(valid_card)
    
del valid_card,temp_df
gc.collect()

21

In [29]:
train.shape

(590540, 519)

## new UID

In [30]:
train['D1_clone'] = train['D1']
test['D1_clone'] = test['D1']
train['D1_clone'].fillna(-999,inplace=True)
test['D1_clone'].fillna(-999,inplace=True)

In [31]:
def first_transactions(x):
    if x['D1_clone']==-999:
        return datetime.date(2000,1,1)
    else:
        periods = x['D1_clone']
        return x['TransactionDateTime'] - pd.Timedelta(days=periods)

In [32]:
%%time
train['First_TransactionDay'] = train.ix[:,['TransactionDateTime','D1_clone']].apply(first_transactions,axis=1)
test['First_TransactionDay'] = test.ix[:,['TransactionDateTime','D1_clone']].apply(first_transactions,axis=1)

CPU times: user 1min 16s, sys: 5.57 s, total: 1min 22s
Wall time: 1min 16s


In [33]:
train['First_TransactionDay'] = train['First_TransactionDay'].dt.date
test['First_TransactionDay'] = test['First_TransactionDay'].dt.date
train.drop(columns='D1_clone',inplace=True)
test.drop(columns='D1_clone',inplace=True)

In [34]:
%%time
train['uid10'] = train['card1'].astype(str)+'_'+train['card2'].astype(str)+'_'+train['card3'].astype(str)+'_'+train['card4'].astype(str)+'_'+\
                train['card5'].astype(str)+'_'+train['card6'].astype(str)+'_'+train['First_TransactionDay'].astype(str)

train['uid11']  = train['card1'].astype(str)+'_'+train['card2'].astype(str)+'_'+train['card3'].astype(str)+'_'+train['card4'].astype(str)+'_'+\
                train['card5'].astype(str)+'_'+train['card6'].astype(str)+'_'+train['First_TransactionDay'].astype(str)+'_'+train['P_emaildomain'].astype(str)



CPU times: user 3.78 s, sys: 745 ms, total: 4.52 s
Wall time: 4.52 s


In [35]:
%%time
test['uid10'] = test['card1'].astype(str)+'_'+test['card2'].astype(str)+'_'+test['card3'].astype(str)+'_'+test['card4'].astype(str)+'_'+\
                test['card5'].astype(str)+'_'+test['card6'].astype(str)+'_'+test['First_TransactionDay'].astype(str)

test['uid11']  = test['card1'].astype(str)+'_'+test['card2'].astype(str)+'_'+test['card3'].astype(str)+'_'+test['card4'].astype(str)+'_'+\
                test['card5'].astype(str)+'_'+test['card6'].astype(str)+'_'+test['First_TransactionDay'].astype(str)+'_'+test['P_emaildomain'].astype(str)

CPU times: user 3.29 s, sys: 569 ms, total: 3.86 s
Wall time: 3.85 s


In [36]:
data = pd.concat([train,test],sort=False,axis=0).reset_index(drop=True)
del train,test
gc.collect()


42

In [37]:
%%time
data['uid10_diff_time_mean'] = data['uid10'].map(data.groupby('uid10')['TransactionDT'].apply(lambda x: np.nanmean(x.diff())).to_dict()) 
data['uid10_diff_time_max'] = data['uid10'].map(data.groupby('uid10')['TransactionDT'].apply(lambda x: np.nanmax(x.diff())).to_dict()) 
data['uid10_diff_time_min'] = data['uid10'].map(data.groupby('uid10')['TransactionDT'].apply(lambda x: np.nanmin(x.diff())).to_dict()) 
        
data['uid11_diff_time_mean'] = data['uid11'].map(data.groupby('uid11')['TransactionDT'].apply(lambda x: np.nanmean(x.diff())).to_dict()) 
data['uid11_diff_time_max'] = data['uid11'].map(data.groupby('uid11')['TransactionDT'].apply(lambda x: np.nanmax(x.diff())).to_dict()) 
data['uid11_diff_time_min'] = data['uid11'].map(data.groupby('uid11')['TransactionDT'].apply(lambda x: np.nanmin(x.diff())).to_dict()) 

CPU times: user 8min 21s, sys: 31.2 s, total: 8min 52s
Wall time: 8min 20s


In [38]:
cols = ['uid10','uid11']
for col in cols:
    valid_card = data[col].value_counts().to_dict()   
    data[col+'_fq_enc'] = data[col].map(valid_card)
    data[col+'_fq_enc']  = data[col].map(valid_card)

In [39]:
del valid_card
gc.collect()


27

In [40]:
data.shape

(1097231, 530)

In [41]:
data['uid10_last_transactiontime'] = data.groupby('uid10')['TransactionDT'].shift(1)
data['uid10_time_gap_between_last_transactionTime'] = data['TransactionDT']-data['uid10_last_transactiontime']

data['uid10_last2_transactiontime'] = data.groupby('uid10')['TransactionDT'].shift(2)
data['uid10_time_gap_between_last_transactionTime'] = data['TransactionDT']-data['uid10_last2_transactiontime']

In [42]:
data['uid11_last_transactiontime'] = data.groupby('uid11')['TransactionDT'].shift(1)
data['uid11_time_gap_between_last_transactionTime'] = data['TransactionDT']-data['uid11_last_transactiontime']

data['uid11_last2_transactiontime'] = data.groupby('uid11')['TransactionDT'].shift(2)
data['uid11_time_gap_between_last_transactionTime'] = data['TransactionDT']-data['uid11_last2_transactiontime']

In [43]:
data.drop(columns=['uid10_last_transactiontime','uid10_last2_transactiontime','uid11_last_transactiontime','uid11_last2_transactiontime'],inplace=True)

In [44]:
temp = data.groupby(['uid10'])['addr1'].shift(1)
data['uid10_last_addr1'] = (data['addr1'] == temp).astype(int)

temp = data.groupby(['uid10'])['addr2'].shift(1)
data['uid10_last_addr2'] = (data['addr2'] == temp).astype(int)


temp = data.groupby(['uid11'])['addr1'].shift(1)
data['uid11_last_addr1'] = (data['addr1'] == temp).astype(int)

temp = data.groupby(['uid11'])['addr2'].shift(1)
data['uid11_last_addr2'] = (data['addr2'] == temp).astype(int)

In [45]:
temp = data.groupby(['uid10'])['ProductCD'].shift(1)
data['uid10_last_ProductCD'] = (data['ProductCD'] == temp).astype(int)

temp = data.groupby(['uid10'])['product_type2'].shift(1)
data['uid10_last_product_type2'] = (data['product_type2'] == temp).astype(int)

temp = data.groupby(['uid11'])['ProductCD'].shift(1)
data['uid11_last_ProductCD'] = (data['ProductCD'] == temp).astype(int)

temp = data.groupby(['uid11'])['product_type2'].shift(1)
data['uid11_last_product_type2'] = (data['product_type2'] == temp).astype(int)

In [46]:
temp = data.groupby(['uid10'])['M4'].shift(1)
data['uid10_last_M4'] = (data['M4'] == temp).astype(int)

temp = data.groupby(['uid11'])['M4'].shift(1)
data['uid11_last_M4'] = (data['M4'] == temp).astype(int)


temp = data.groupby(['uid10'])['TransactionAmt'].shift(1)
data['uid10_last_TransactionAmt'] = (data['TransactionAmt'] == temp).astype(int)


temp = data.groupby(['uid11'])['TransactionAmt'].shift(1)
data['uid11_last_TransactionAmt'] = (data['TransactionAmt'] == temp).astype(int)

In [47]:
temp = data.groupby(['uid10'])['P_emaildomain'].shift(1)
data['uid10_last_P_emaildomain'] = (data['P_emaildomain'] == temp).astype(int)

temp = data.groupby(['uid10'])['R_emaildomain'].shift(1)
data['uid10_last_R_emaildomain'] = (data['R_emaildomain'] == temp).astype(int)


temp = data.groupby(['uid11'])['R_emaildomain'].shift(1)
data['uid11_last_R_emaildomain'] = (data['R_emaildomain'] == temp).astype(int)

In [48]:
for col in "card1,card2,card5,addr1,addr2".split(","):
    col_count1 = data[data['C3'] == 0].groupby(col)['C3'].count()
    col_count2 = data[data['C3'] != 0].groupby(col)['C3'].count()
    data[col+'_C3count'] = data[col].map(col_count2) / (data[col].map(col_count1) + 0.01)
    data[col+'_C3count'] = data[col].map(col_count2) / (data[col].map(col_count1) + 0.01)
    
    col_count1 = data[data['C5'] == 0].groupby(col)['C5'].count()
    col_count2 = data[data['C5'] != 0].groupby(col)['C5'].count()
    data[col+'_C5count'] = data[col].map(col_count2) / (data[col].map(col_count1) + 0.01)
    data[col+'_C5count'] = data[col].map(col_count2) / (data[col].map(col_count1) + 0.01)
    
    col_count1 = data[data['C9'] == 0].groupby(col)['C9'].count()
    col_count2 = data[data['C9'] != 0].groupby(col)['C9'].count()
    data[col+'_C9count'] = data[col].map(col_count2) / (data[col].map(col_count1) + 0.01)
    data[col+'_C9count'] = data[col].map(col_count2) / (data[col].map(col_count1) + 0.01)


In [49]:
del temp
gc.collect()

361

In [50]:
train = data.iloc[:nrows]
test = data.iloc[nrows:]

In [51]:
temp_df = pd.concat([train[['uid3','uid4','uid7','uid10','uid11','ProductCD']],test[['uid3','uid4','uid7','uid10','uid11','ProductCD']]])
cols= ['uid3','uid4','uid7','uid10','uid11']
for col in cols:
    new_col = col+'_'+'ProductCD_nunique'
    _ = temp_df.groupby(col)['ProductCD'].nunique().reset_index()
    _.columns=[col,new_col]
    train = pd.merge(train,_,on=col,how='left')
    test = pd.merge(test,_,on=col,how='left')

del temp_df,_
gc.collect()

42

In [52]:
i_cols = ['card1','card2','card3','card4','card5','uid3','uid4','uid7','uid10','uid11']

for col in i_cols:
    for agg_type in ['mean','std','max','sum','min']:
        new_col_name = col+'_TransactionAmt_'+agg_type
        temp_df = pd.concat([train[[col, 'TransactionAmt']], test[[col,'TransactionAmt']]])
        temp_df = temp_df.groupby([col])['TransactionAmt'].agg([agg_type]).reset_index().rename(
                                                columns={agg_type: new_col_name})
        temp_df.index = list(temp_df[col])
        temp_df = temp_df[new_col_name].to_dict()   
    
        train[new_col_name] = train[col].map(temp_df)
        test[new_col_name]  = test[col].map(temp_df)
        
    train[col+'_TransactionAmt_mean_std'] =  train[col+'_TransactionAmt_mean'] / train[col+'_TransactionAmt_std']
    test[col+'_TransactionAmt_mean_std']  = test[col+'_TransactionAmt_mean'] / test[col+'_TransactionAmt_std']

In [53]:
del temp_df,mask
gc.collect()

48

In [54]:
del data
gc.collect()

27

### Timeblock frequency encoding

In [55]:
train['D_Max'] = train[d_cols].max(axis=1)
test['D_Max'] = test[d_cols].max(axis=1)

In [56]:
d_cols = ['D'+str(i) for i in range(1,16)]
d_cols.append('D_Max')

In [57]:
periods = ['TransactionDateTime_month','TransactionDateTime_week','TransactionDateTime_day']
for df in [train, test]:
    df = values_normalization(df, periods, d_cols)

In [58]:
aggregations = ['mean','std','max','min','sum']
uids = ['uid10','uid11']
i_cols = ['D'+str(i)+"_TransactionDateTime_day_min_max" for i in range(1,16)]
i_cols.append('D_Max_TransactionDateTime_day_min_max')


In [59]:
%%time
train, test = uid_aggregation(train, test, i_cols, uids, aggregations)

CPU times: user 21min 20s, sys: 24min 43s, total: 46min 3s
Wall time: 18min 20s


In [60]:
train.shape,test.shape

((590540, 884), (506691, 884))

In [62]:
train_identity = pd.read_csv('./input/train_identity.csv')
test_identity = pd.read_csv('./input/test_identity.csv')

In [64]:
def id_split(dataframe):
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]
    
    dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
    dataframe['version_id_30'] = dataframe['id_30'].astype(str).apply(lambda x: x.split(' ')[-1])

    dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id_31'].astype(str).apply(lambda x: x.split(' ')[-1])

    dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
    dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]

    dataframe['id_34'] = dataframe['id_34'].str.split(':', expand=True)[1]
    dataframe['id_23'] = dataframe['id_23'].str.split(':', expand=True)[1]

    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    dataframe['had_id'] = 1
    gc.collect()
    
    return dataframe

In [65]:
train_identity = id_split(train_identity)
test_identity = id_split(test_identity)

In [76]:
print('Merging data...')
train = pd.merge(train,train_identity, on='TransactionID', how='left')
test = pd.merge(test,test_identity, on='TransactionID', how='left')

print('Data was successfully merged!\n')

del train_identity, test_identity

print(f'Train dataset has {train.shape[0]} rows and {train.shape[1]} columns.')
print(f'Test dataset has {test.shape[0]} rows and {test.shape[1]} columns.\n')

gc.collect()

Merging data...
Data was successfully merged!

Train dataset has 590540 rows and 933 columns.
Test dataset has 506691 rows and 933 columns.



93

In [77]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,...,uid11_D_Max_TransactionDateTime_day_min_max_sum,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,device_name,device_version,OS_id_30,version_id_30,browser_id_31,version_id_31,screen_width,screen_height,had_id
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,...,0.011256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,...,0.000670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,T,T,T,M0,...,1.299285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN,M0,...,9.598221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,Samsung,NRD90M,Android,7.0,samsung,6.2,2220,1080,1.0


In [78]:
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
test['id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
test['id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')

train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

In [79]:
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    train[feature] = train[f1].astype(str) + '_' + train[f2].astype(str)
    test[feature] = test[f1].astype(str) + '_' + test[f2].astype(str)

    le = LabelEncoder()
    le.fit(list(train[feature].astype(str).values) + list(test[feature].astype(str).values))
    train[feature] = le.transform(list(train[feature].astype(str).values))
    test[feature] = le.transform(list(test[feature].astype(str).values))


In [80]:
for feature in ['id_36']:
    train[feature + '_count_full'] = train[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
    test[feature + '_count_full'] = test[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))

In [81]:
for feature in ['id_01', 'id_31', 'id_33', 'id_36']:
    train[feature + '_count_dist'] = train[feature].map(train[feature].value_counts(dropna=False))
    test[feature + '_count_dist'] = test[feature].map(test[feature].value_counts(dropna=False))

In [82]:
train["lastest_browser"] = np.zeros(train.shape[0])
test["lastest_browser"] = np.zeros(test.shape[0])

def setBrowser(df):
    df.loc[df["id_31"]=="samsung browser 7.0",'lastest_browser']=1
    df.loc[df["id_31"]=="opera 53.0",'lastest_browser']=1
    df.loc[df["id_31"]=="mobile safari 10.0",'lastest_browser']=1
    df.loc[df["id_31"]=="google search application 49.0",'lastest_browser']=1
    df.loc[df["id_31"]=="firefox 60.0",'lastest_browser']=1
    df.loc[df["id_31"]=="edge 17.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 69.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 67.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 63.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 63.0 for ios",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 64.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 64.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 64.0 for ios",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 65.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 65.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 65.0 for ios",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 66.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 66.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 66.0 for ios",'lastest_browser']=1
    return df

train=setBrowser(train)
test=setBrowser(test)

In [83]:
def setDevice(df):
    df['DeviceInfo'] = df['DeviceInfo'].fillna('unknown_device').str.lower()
    
    df['device_name'] = df['DeviceInfo'].str.split('/', expand=True)[0]

    df.loc[df['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    df.loc[df['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    df.loc[df['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    df.loc[df['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    df.loc[df['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    df.loc[df['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    df.loc[df['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    df.loc[df['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    df.loc[df['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    df.loc[df['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    df.loc[df['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    df.loc[df['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    df.loc[df['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    df.loc[df['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    df.loc[df['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    df.loc[df['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    df.loc[df['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    df.loc[df.device_name.isin(df.device_name.value_counts()[df.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    df['had_id'] = 1
    gc.collect()
    
    return df

train=setDevice(train)
test=setDevice(test)

In [84]:
i_cols = ['id_30','id_33']

for col in i_cols:
    temp_df = pd.concat([train[[col]], test[[col]]])
    fq_encode = temp_df[col].value_counts(dropna=False).to_dict()   
    train[col+'_fq_enc'] = train[col].map(fq_encode)
    test[col+'_fq_enc']  = test[col].map(fq_encode)

In [87]:
train.to_pickle('./output/train_transaction_update.pkl')
test.to_pickle('./output/test_transaction_update.pkl')

In [1]:
from IPython.display import FileLink, FileLinks
FileLinks('./output/')


./output/
  test_transaction_update_1.pkl
  test_transaction_update.pkl
  train_transaction_update_1.pkl
  train_transaction_update.pkl
  last_transaction.pkl
  transaction_identity.pkl
  Groups.pkl

In [89]:
train.shape,test.shape

((590540, 965), (506691, 965))

In [90]:
cols = [i for i in train.columns if i not in 'isFraud']
train[cols].shape,test[cols].shape


((590540, 964), (506691, 964))

In [91]:
[i for i in test.columns if i not in train.columns]

[]

In [92]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,device_name,device_version,OS_id_30,version_id_30,browser_id_31,version_id_31,screen_width,screen_height,had_id,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,TransactionAmt_to_std_card4,id_02_to_mean_card1,id_02_to_mean_card4,id_02_to_std_card1,id_02_to_std_card4,D15_to_mean_card1,D15_to_mean_card4,D15_to_std_card1,D15_to_std_card4,D15_to_mean_addr1,D15_to_std_addr1,id_02__id_20,id_02__D8,D11__DeviceInfo,DeviceInfo__P_emaildomain,P_emaildomain__C2,card2__dist1,card1__card5,card2__id_20,card5__P_emaildomain,addr1__card1,id_36_count_full,id_01_count_dist,id_31_count_dist,id_33_count_dist,id_36_count_dist,lastest_browser,id_30_fq_enc,id_33_fq_enc
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown_device,unknown_device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.194640,0.257761,0.184566,0.170233,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270749,259912,35,8523,4686,31072,4648,9239,508,29248,819269,446307,450258,517251,449555,0.0,949007,953271
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown_device,unknown_device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.123777,0.219054,0.062995,0.114212,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270749,259912,3620,8507,1589,18129,10936,5273,43,32205,819269,446307,450258,517251,449555,0.0,949007,953271
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0,T,T,T,M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown_device,unknown_device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.608150,0.443070,0.589241,0.258544,NaN,NaN,NaN,NaN,2.518583,1.865898,1.851016,1.541458,1.611525,1.486472,270749,259912,241,8527,5141,23016,12999,6795,750,35560,819269,446307,450258,517251,449555,0.0,949007,953271
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN,M0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown_device,unknown_device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.405133,0.377679,0.259447,0.196917,NaN,NaN,NaN,NaN,0.550272,0.720046,0.520531,0.558397,0.686169,0.576638,270749,259912,3620,8544,6495,29394,9636,8638,170,47566,819269,446307,450258,517251,449555,0.0,949007,953271
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32.0,2220x1080,2,T,F,T,T,mobile,samsung sm-g892a build/nrd90m,Others,NRD90M,Android,7.0,samsung,6.2,2220,1080,1,0.515612,0.377679,0.882933,0.196917,0.764773,0.373295,1.753301,0.425884,NaN,NaN,NaN,NaN,NaN,NaN,229802,221147,2445,4375,1589,25501,12822,7169,43,40560,267353,19555,1061,544,134066,0.0,3573,1430
